# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import time
import gmaps
from pprint import pprint
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

586

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [3]:
weather_api_key

'c51718a8fe1f408977826bbb65188383'

In [4]:
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"
query_url = f"{url}appid={weather_api_key}&units={units}&q="

In [5]:
#containers for data
city_names = []
countries = []
latitudes = []
longitudes = []
temperatures = []
humidities = []
cloudy_skies = []
windiness = []

In [6]:
for city in cities:
    try:
        #get the data into json format
        response = requests.get(query_url + city).json()
        
        #store data into lists
        city_names.append(response['name'])
        latitudes.append(response['coord']['lat'])
        longitudes.append(response['coord']['lon'])
        cloudy_skies.append(response["clouds"]["all"])
        windiness.append(response["wind"]["speed"])
        temperatures.append(response["main"]["temp_max"])
        humidities.append(response["main"]["humidity"])
        countries.append(response['sys']['country'])
        print(f"{city} information is processed")
        print("-" * 30)
    except:
        continue
        #cities not listed in the output were not found in the API
       

east london information is processed
------------------------------
lagoa information is processed
------------------------------
oyem information is processed
------------------------------
yellowknife information is processed
------------------------------
ahipara information is processed
------------------------------
narsaq information is processed
------------------------------
mataura information is processed
------------------------------
barrow information is processed
------------------------------
caravelas information is processed
------------------------------
albany information is processed
------------------------------
samarai information is processed
------------------------------
batemans bay information is processed
------------------------------
khatanga information is processed
------------------------------
bluff information is processed
------------------------------
lebu information is processed
------------------------------
puerto ayora information is processed

murray bridge information is processed
------------------------------
chilca information is processed
------------------------------
aswan information is processed
------------------------------
maniitsoq information is processed
------------------------------
hollister information is processed
------------------------------
bam information is processed
------------------------------
dawei information is processed
------------------------------
aberystwyth information is processed
------------------------------
luhyny information is processed
------------------------------
san patricio information is processed
------------------------------
bucerias information is processed
------------------------------
harper information is processed
------------------------------
kenai information is processed
------------------------------
la palma information is processed
------------------------------
berbera information is processed
------------------------------
birao information is processed
-

qaanaaq information is processed
------------------------------
grootfontein information is processed
------------------------------
fare information is processed
------------------------------
rajmahal information is processed
------------------------------
riyadh information is processed
------------------------------
yining information is processed
------------------------------
clyde river information is processed
------------------------------
phayao information is processed
------------------------------
wajima information is processed
------------------------------
tonder information is processed
------------------------------
comodoro rivadavia information is processed
------------------------------
deer lake information is processed
------------------------------
buta information is processed
------------------------------
imbituba information is processed
------------------------------
vila franca do campo information is processed
------------------------------
marawi informa

kearney information is processed
------------------------------
thanh hoa information is processed
------------------------------
saint-pierre-des-corps information is processed
------------------------------
gravelbourg information is processed
------------------------------
channel-port aux basques information is processed
------------------------------
gimli information is processed
------------------------------
aksu information is processed
------------------------------
wailuku information is processed
------------------------------
tizapan el alto information is processed
------------------------------
yeppoon information is processed
------------------------------
taltal information is processed
------------------------------
mwene-ditu information is processed
------------------------------
saint-francois information is processed
------------------------------
esperance information is processed
------------------------------
luwingu information is processed
-------------------

hasaki information is processed
------------------------------
kavieng information is processed
------------------------------
oranjemund information is processed
------------------------------
padang information is processed
------------------------------
fortuna information is processed
------------------------------
carlsbad information is processed
------------------------------
zheleznodorozhnyy information is processed
------------------------------
beira information is processed
------------------------------
wood river information is processed
------------------------------
san information is processed
------------------------------
nelson bay information is processed
------------------------------
fiumicino information is processed
------------------------------
sinnamary information is processed
------------------------------
kintampo information is processed
------------------------------
lebork information is processed
------------------------------
bay roberts information 

# Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [7]:
weather_df = pd.DataFrame({
    "City": city_names,
    "Country": countries,
    "Latitude":latitudes,
    "Longitude": longitudes,
    "Temperatures": temperatures,
    "Humidity": humidities,
    "Cloudiness": cloudy_skies,
    "Windiness" : windiness
})
weather_df.head()

,City,Country,Latitude,Longitude,Temperatures,Humidity,Cloudiness,Windiness
0,East London,ZA,-33.0153,27.9116,66.20,94,75,9.22
1,Lagoa,PT,39.0500,-27.9833,58.19,82,33,26.31
2,Oyem,GA,1.5995,11.5793,68.22,97,100,1.57
3,Yellowknife,CA,62.4560,-114.3525,-35.00,67,75,9.22
4,Ahipara,NZ,-35.1667,173.1667,72.32,60,56,9.04


In [8]:
weather_df.to_csv("./Data/weather_info.csv")

In [9]:
weather_df.count()

City            546
Country         546
Latitude        546
Longitude       546
Temperatures    546
Humidity        546
Cloudiness      546
Windiness       546
dtype: int64

Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [10]:
#  Get the indices of cities that have humidity over 100%.
weather_df[weather_df["Humidity"] > 100]

,City,Country,Latitude,Longitude,Temperatures,Humidity,Cloudiness,Windiness


In [11]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".
print(f"There are no cities with humidities > 100%")

There are no cities with humidities > 100%


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression